*This is an example for sequence alignment using and edit distance algorithm.*

In [2]:
# I believe your data is more-or-less in the following format:

# you have a long string for the text (let's presume you've already converted it to UTF-8)
text_reference = "In Deutschland sind etwa 1 Mio. Menschen chronisch mit Hepatitisviren der Gruppen B und C infiziert, die Mehrzahl davon im reproduktionsfähigen Alter. Bei Kinderwunsch ist neben der hepatologischen Situation insbesondere das sexuelle und das maternofetale Übertragungsrisiko zu berücksichtigen. Bei Hepatitis B kann durch Vakzinierung die sexuelle Transmission vermieden und durch neonatale Simultanimpfung die maternofetale Übertragungsrate drastisch gesenkt werden, die Langzeitprognose konnatal infizierter Kinder ist eher günstig. Bei chronischer Infektion kann derzeit aber die Möglichkeit einer paternofetalen Virustransmission durch infizierte Spermien nicht ausgeschlossen werden. Da für Hepatitis C weder eine Vakzinierung noch eine Immunprophylaxe verfügbar ist, erfordert die Prävention der sexuellen Übertragung den Gebrauch von Kondomen. Bei Infektion des (Ehe-)Mannes kann zur Risikoreduktion eine ART mit aufbereiteten Spermien angeboten werden. Bei Infektion der (Ehe-)Frau lassen das geringe maternofetale Übertragungsrisiko und die jahrzehntelang günstige Prognose konnatal infizierter Kinder keine Kontraindikation gegen ART erkennen. Vor jeder reproduktionsmedizinischen Intervention sollte aber die Option einer viruseliminierenden Therapie geprüft werden."
# you have a list of sentences which contains a list of words from the xml file. This is UTF-8 but has been altered in some ways (see below)
xml_sentences = [["In", "Deutschland", "sind", "etwa", "1", "Mio", "."], ["Menschen", "chronisch", "mit", "Hepatitisviren", "der", "Gruppen", "B", "und", "C", "infiziert", ",", "die", "Mehrzahl", "davon", "im", "reproduktionsfaehigen", "Alter","."], ["Bei", "Kinderwunsch", "ist", "neben", "der", "hepatologischen", "Situation", "insbesondere", "das", "sexuelle", "und", "das", "maternofetale", "Uebertragungsrisiko", "zu", "beruecksichtigen", "."], ["Bei", "Hepatitis", "B", "kann", "durch", "Vakzinierung", "die", "sexuelle", "Transmission", "vermieden", "und", "durch", "neonatale", "Simultanimpfung", "die", "maternofetale", "Uebertragungsrate", "drastisch", "gesenkt", "werden", ",", "die", "Langzeitprognose", "konnatal", "infizierter", "Kinder", "ist", "eher", "guenstig", "."], ["Bei", "chronischer", "Infektion", "kann", "derzeit", "aber", "die", "Moeglichkeit", "einer", "paternofetalen", "Virustransmission", "durch", "infizierte", "Spermien", "nicht", "ausgeschlossen", "werden", "."], ["Da", "fuer", "Hepatitis", "C", "weder", "eine", "Vakzinierung", "noch", "eine", "Immunprophylaxe", "verfuegbar", "ist", ",", "erfordert", "die", "Praevention", "der", "sexuellen", "Uebertragung", "den", "Gebrauch", "von", "Kondomen", "."], ["Bei", "Infektion", "des", "Ehe-", ")", "Mannes", "(", "kann", "zur", "Risikoreduktion", "eine", "ART", "mit", "aufbereiteten", "Spermien", "angeboten", "werden", "."], ["Bei", "Infektion", "der", "Ehe-", ")", "Frau", "(", "lassen", "das", "geringe", "maternofetale", "Uebertragungsrisiko", "und", "die", "jahrzehntelang", "guenstige", "Prognose", "konnatal", "infizierter", "Kinder", "keine", "Kontraindikation", "gegen", "ART", "erkennen", "."], ["Vor", "jeder", "reproduktionsmedizinischen", "Intervention", "sollte", "aber", "die", "Option", "einer", "viruseliminierenden", "Therapie", "geprueft", "werden", "."]]
## interesting note on the side: there's an error in the XML: of course, the sentence does not finish after saying "1 Mio." but that's the abbreviation for "1 million". 
## note that we won't fix such mistakes -- it's wrong but we'll live with it. (Maybe this is then fixed by having multiple sentences aligned to one via the xml annotations.)

Description of the approach:

* we'll create a character-by-character alignment between the "original" text and the text from XML,
* from the alignment, we can infer the (original) text that corresponds to each (xml) sentence.

to optimize this, we can assign insertion/deletion costs that are specific to our data. For example:
* low cost for deleting spaces (as, for simplicity, we won't put any whitespace in between the xml tokens).
* very low cost for substituting ( with ) (and vice-versa) as that's what we find in the data.
* low cost for substituting äöüÄÖÜ against aouAOU and a low cost for inserting e (of course, it would be ideal to have a low cost for substituting ä with ae but multi-character substitutions are probably beyond what we want to deal with...

→ for the moment we'll simply try without any of these optimizations. Instead we'll simply say that deletions are slightly cheaper than normal.

In [3]:
import brew_distance

In [33]:
xml_text = ''.join(sum(xml_sentences, start=[]))

In [35]:
cost, edits = brew_distance.distance(text_reference, xml_text, "both", cost=(0, 1, 0.8, 1))
rel_cost = cost / len(text_reference)
if (rel_cost > 0.2):
    print("these are really different: ", (cost, rel_cost, text_reference, xml_text))
#rel_cost, edits

In [57]:
sentence_counter = 0
text_sentence_start = 0
sentences = []

xml_sentence_lengths = [len(''.join(sent)) for sent in xml_sentences]
xml_sentence_ends = [0] * len(xml_sentence_lengths) + [-1]
for i in range(len(xml_sentence_lengths)):
    xml_sentence_ends[i] = sum(xml_sentence_lengths[0:i+1]) - 1
#print(xml_sentence_lengths, xml_sentence_ends, len(xml_text), sum(xml_sentence_lengths))
text_counter, xml_counter = 0, 0
for edit in edits:
    if edit in ("MATCH", "SUBST"):
        text_counter += 1
        xml_counter += 1
    elif edit == "INS":
        xml_counter += 1
    elif edit == "DEL": 
        text_counter += 1
    else:
        raise AssertionError("unknown edit type" + edit)
    if xml_counter == xml_sentence_ends[sentence_counter]:
        sentences += [text_reference[text_sentence_start:text_counter+1].lstrip()]
        text_sentence_start = text_counter + 1
        sentence_counter += 1

In [58]:
print(sentences)

['In Deutschland sind etwa 1 Mio.', 'Menschen chronisch mit Hepatitisviren der Gruppen B und C infiziert, die Mehrzahl davon im reproduktionsfähigen Alter.', 'Bei Kinderwunsch ist neben der hepatologischen Situation insbesondere das sexuelle und das maternofetale Übertragungsrisiko zu berücksichtigen.', 'Bei Hepatitis B kann durch Vakzinierung die sexuelle Transmission vermieden und durch neonatale Simultanimpfung die maternofetale Übertragungsrate drastisch gesenkt werden, die Langzeitprognose konnatal infizierter Kinder ist eher günstig.', 'Bei chronischer Infektion kann derzeit aber die Möglichkeit einer paternofetalen Virustransmission durch infizierte Spermien nicht ausgeschlossen werden.', 'Da für Hepatitis C weder eine Vakzinierung noch eine Immunprophylaxe verfügbar ist, erfordert die Prävention der sexuellen Übertragung den Gebrauch von Kondomen.', 'Bei Infektion des (Ehe-)Mannes kann zur Risikoreduktion eine ART mit aufbereiteten Spermien angeboten werden.', 'Bei Infektion de